<a id="top"></a>
# Downloading UV data – Part 1 – SOLUTIONS
***

Solutions proposed for Part 1.

## Imports

The first step will be to import the libraries we will be using throughout this tutorial: 

- `Observations` from _astroquery.mast_ to query the Barbara A. Mikulski Archive for Space Telescopes (MAST).
- `fits` from _astropy.io_ for accessing FITS files
- `numpy` for array manipulations

In [ ]:
from astroquery.mast import Observations
from astropy.io import fits
import numpy as np

## Functions needed

Functions defined in the tutorial that will be used in the solutions of the exercises:

In [ ]:
def changeNaN(wavv,fluxx,wavv1,fluxx1):
    
    """
    Removes negative flux values to avoid encountering NaN when converting to flux scale
    
    Inputs:
    :param wavv,wavv1: input wavelength arrays
    :type wavv,wavv1: float array
    :param fluxx,fluxx1: input wavelength arrays
    :type fluxx,fluxx1: float array
    
    Outputs: 
    :return: wavv,fluxx,wavv1,fluxx1 without the corresponding problematic values
    :rtype: float array
    """
    
    prob = np.where(fluxx < 0)[0]
    wavv = np.delete(wavv,prob)
    fluxx = np.delete(fluxx,prob)
    wavv1 = np.delete(wavv1,prob)
    fluxx1 = np.delete(fluxx1,prob)
    
    prob2 = np.where(fluxx1 < 0)[0]
    wavv = np.delete(wavv,prob2)
    fluxx = np.delete(fluxx,prob2)
    wavv1 = np.delete(wavv1,prob2)
    fluxx1 = np.delete(fluxx1,prob2)
    
    return wavv, fluxx, wavv1, fluxx1

In [ ]:
def saveArray(array,filename):
    
    """
    Saves arrays into .txt files in current directory and checks that the files have been written properly
    
    Inputs:
    :param array: array to be saved to local directory
    :type array: float array
    :param filename: name of file to be saved
    :type filename: string
    
    Outputs: 
    :return: none
    :rtype: none
    """
    
    # First display the start and ending of the array to check later
    print('First 3 elements of original array:\n', array[:3])
    print('Last 3 elements of original array:\n', array[-3:])

    # Save the array in a text file
    np.savetxt(filename,array)
    
    # Display the saved content to check it was correctly written to the .txt file
    content = np.loadtxt(filename)
    print('First 3 elements of '+filename+':\n', content[:3])
    print('Last 3 elements of '+filename+':\n', content[-3:],'\n')
    
    return None

## Exercises

Now you can try to do it yourself! Try to obtain the spectra necessary to create an extinction curve of a star belonging to the Large Magellanic Cloud (LMC) following the steps presented for the SMC one.

### 1. Load the data

The targets for this exercise will be:

- Sk -69 206 (Observation IDs: swp36552, lwp15751) (reddened star)
- Sk -67 5 (Observation IDs: swp04827, lwr04170) (unreddened star)

First, do the database search for the reddened star:

In [ ]:
IUEobs_sk_69_206 = Observations.query_criteria(objectname="SK-69206",obs_collection="IUE")
data_products_sk_69_206 = Observations.get_product_list(IUEobs_sk_69_206)

data_products_sk_69_206

Narrow down the possibilities to the desired observation, observation ID: "swp36552":

In [ ]:
IUEobs_sk_69_206_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp36552")
data_products_sk_69_206_sw = Observations.get_product_list(IUEobs_sk_69_206_sw)
yourProd_sk_69_206_sw = Observations.filter_products(data_products_sk_69_206_sw, extension="swp36552mxlo_vo.fits")

yourProd_sk_69_206_sw

Now, download the data as before:

In [ ]:
Observations.download_products(yourProd_sk_69_206_sw, mrp_only = False, cache = False) 

And again, in order to cover a wider wavelength range, let's also download the other part of the spectra using the second observation ID provided, observation ID: "lwp15751":

In [ ]:
IUEobs_sk_69_206_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwp15751")
data_products_sk_69_206_lw = Observations.get_product_list(IUEobs_sk_69_206_lw)
yourProd_sk_69_206_lw = Observations.filter_products(data_products_sk_69_206_lw, extension="lwp15751mxlo_vo.fits")
Observations.download_products(yourProd_sk_69_206_lw, mrp_only = False, cache = False) 

Now, let's do the same for the unreddened star:

In [ ]:
IUEobs_sk_67_5 = Observations.query_criteria(objectname="SK-675",obs_collection="IUE")
data_products_sk_67_5 = Observations.get_product_list(IUEobs_sk_67_5)

data_products_sk_67_5

- Observation ID: "swp04827":

In [ ]:
data_products_sk_67_5[42]

In [ ]:
IUEobs_sk_67_5_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp04827")
data_products_sk_67_5_sw = Observations.get_product_list(IUEobs_sk_67_5_sw)
yourProd_sk_67_5_sw = Observations.filter_products(data_products_sk_67_5_sw, extension="swp04827mxlo_vo.fits")
Observations.download_products(yourProd_sk_67_5_sw, mrp_only = False, cache = False) 

- Observation ID: "lwr04170"

In [ ]:
data_products_sk_67_5[17]

In [ ]:
IUEobs_sk_67_5_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwr04170")
data_products_sk_67_5_lw = Observations.get_product_list(IUEobs_sk_67_5_lw)
yourProd_sk_67_5_lw = Observations.filter_products(data_products_sk_67_5_lw, extension="lwr04170mxlo_vo.fits")
Observations.download_products(yourProd_sk_67_5_lw, mrp_only = False, cache = False) 

### 2. Open the fits file and explore it

In [ ]:
filename_sk_69_206_sw = "./mastDownload/IUE/swp36552/swp36552mxlo_vo.fits"
fits.info(filename_sk_69_206_sw)

with fits.open(filename_sk_69_206_sw) as hdulist:
    spectrum_sk_69_206_sw = hdulist[1].data

wav_sk_69_206_sw = spectrum_sk_69_206_sw[0][0] # A
flux_sk_69_206_sw = spectrum_sk_69_206_sw[0][1] # ergs/cm2/sec/A

filename_sk_69_206_lw = "./mastDownload/IUE/lwp15751/lwp15751mxlo_vo.fits"
fits.info(filename_sk_69_206_lw)

with fits.open(filename_sk_69_206_lw) as hdulist:
    spectrum_sk_69_206_lw = hdulist[1].data

wav_sk_69_206_lw = spectrum_sk_69_206_lw[0][0] # A
flux_sk_69_206_lw = spectrum_sk_69_206_lw[0][1] # ergs/cm2/sec/A

filename_sk_67_5_sw = "./mastDownload/IUE/swp04827/swp04827mxlo_vo.fits"
fits.info(filename_sk_67_5_sw)

with fits.open(filename_sk_67_5_sw) as hdulist:
    spectrum_sk_67_5_sw = hdulist[1].data

wav_sk_67_5_sw = spectrum_sk_67_5_sw[0][0] # A
flux_sk_67_5_sw = spectrum_sk_67_5_sw[0][1] # ergs/cm2/sec/A

filename_sk_67_5_lw = "./mastDownload/IUE/lwr04170/lwr04170mxlo_vo.fits"
fits.info(filename_sk_67_5_lw)

with fits.open(filename_sk_67_5_lw) as hdulist:
    spectrum_sk_67_5_lw = hdulist[1].data

wav_sk_67_5_lw = spectrum_sk_67_5_lw[0][0] # A
flux_sk_67_5_lw = spectrum_sk_67_5_lw[0][1] # ergs/cm2/sec/A

wav_sk_69_206_sw, flux_sk_69_206_sw, wav_sk_67_5_sw, flux_sk_67_5_sw = changeNaN(wav_sk_69_206_sw, flux_sk_69_206_sw, wav_sk_67_5_sw, flux_sk_67_5_sw)
wav_sk_69_206_lw, flux_sk_69_206_lw, wav_sk_67_5_lw, flux_sk_67_5_lw = changeNaN(wav_sk_69_206_lw, flux_sk_69_206_lw, wav_sk_67_5_lw, flux_sk_67_5_lw)

### 3. Save the data to your local computer

In [ ]:
arrays = [wav_sk_69_206_sw, flux_sk_69_206_sw, wav_sk_67_5_sw, flux_sk_67_5_sw, wav_sk_69_206_lw, flux_sk_69_206_lw, wav_sk_67_5_lw, flux_sk_67_5_lw]
filenames = ['wav_sk_69_206_sw.txt', 'flux_sk_69_206_sw.txt', 'wav_sk_67_5_sw.txt', 'flux_sk_67_5_sw.txt', 'wav_sk_69_206_lw.txt', 'flux_sk_69_206_lw.txt', 'wav_sk_67_5_lw.txt', 'flux_sk_67_5_lw.txt']

for i in range(len(arrays)):
    saveArray(arrays[i],filenames[i])

## Additional Resources

For more information about the MAST archive and details about mission data: 

* <a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
* <a href="https://archive.stsci.edu/iue/">International Ultraviolet Explorer Page (MAST)</a> <br>
* <a href="https://archive.stsci.edu/iue/mdr_help.html">IUE Data Retrieval Help Page</a> 

For more information about extinction curves and their parametrization:


* <a href="https://ui.adsabs.harvard.edu/abs/2003ApJ...594..279G/abstract">A quantitative comparison of SMC, LMC and Milky Way UV to NIR extinction curves</a> <br>
* <a href="https://adsabs.harvard.edu/pdf/1990ApJS...72..163F">An analysis of the shapes of the ultraviolet extinction curves. III. An atlas of ultraviolet extinction curves</a> 

## About this Notebook

**Author**: Clara Puerto Sánchez <br>
**Keyword(s)**: Tutorial, UV, reddening, extinction-curve <br>
**Last Updated**: Aug 2022 <br>
**Next review**: Mar 2023

For support, please contact the Archive HelpDesk at archive@stsci.edu.
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>